# Conser-vision Practice Area: Image Classification

## General imports

In [2]:
import cv2
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
import os
import shutil
import random
import numpy as np 
import pandas as pd 
import os
import torch
import torchvision
from torchvision import datasets, transforms
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, sampler, random_split
from torchvision import models
from tqdm import tqdm
import time
import copy
import timm
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import timm
from PIL import Image
import os
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device


/home/user/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device(type='cuda')

# Download the necessary files from the competition. <br> The is provided in problem description.

# Train Features

In [3]:
df  = pd.read_csv("train_features.csv")
df.head()

,id,filepath,site
0,ZJ000000,train_features/ZJ000000.jpg,S0120
1,ZJ000001,train_features/ZJ000001.jpg,S0069
2,ZJ000002,train_features/ZJ000002.jpg,S0009
3,ZJ000003,train_features/ZJ000003.jpg,S0008
4,ZJ000004,train_features/ZJ000004.jpg,S0036


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16488 entries, 0 to 16487
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        16488 non-null  object
 1   filepath  16488 non-null  object
 2   site      16488 non-null  object
dtypes: object(3)
memory usage: 386.6+ KB


In [5]:
unique_sites_train = sorted(df['site'].unique())
print(unique_sites_train)
print(len(unique_sites_train))

['S0001', 'S0002', 'S0003', 'S0004', 'S0005', 'S0006', 'S0007', 'S0008', 'S0009', 'S0010', 'S0013', 'S0014', 'S0015', 'S0016', 'S0017', 'S0018', 'S0019', 'S0020', 'S0021', 'S0022', 'S0023', 'S0024', 'S0025', 'S0026', 'S0027', 'S0028', 'S0029', 'S0030', 'S0031', 'S0032', 'S0035', 'S0036', 'S0038', 'S0042', 'S0043', 'S0044', 'S0045', 'S0046', 'S0047', 'S0049', 'S0050', 'S0051', 'S0053', 'S0054', 'S0056', 'S0059', 'S0060', 'S0061', 'S0062', 'S0063', 'S0068', 'S0069', 'S0070', 'S0071', 'S0073', 'S0074', 'S0075', 'S0076', 'S0077', 'S0078', 'S0079', 'S0080', 'S0081', 'S0083', 'S0084', 'S0085', 'S0088', 'S0089', 'S0092', 'S0093', 'S0094', 'S0095', 'S0096', 'S0097', 'S0098', 'S0101', 'S0102', 'S0104', 'S0105', 'S0106', 'S0107', 'S0108', 'S0110', 'S0112', 'S0113', 'S0115', 'S0117', 'S0119', 'S0120', 'S0121', 'S0122', 'S0123', 'S0124', 'S0125', 'S0127', 'S0129', 'S0130', 'S0131', 'S0133', 'S0134', 'S0136', 'S0137', 'S0138', 'S0139', 'S0141', 'S0143', 'S0144', 'S0146', 'S0147', 'S0148', 'S0149', 

In [6]:
unique_id_train = sorted(df['id'].unique())
print(unique_id_train)
print(len(unique_id_train))

['ZJ000000', 'ZJ000001', 'ZJ000002', 'ZJ000003', 'ZJ000004', 'ZJ000005', 'ZJ000006', 'ZJ000007', 'ZJ000008', 'ZJ000009', 'ZJ000010', 'ZJ000011', 'ZJ000012', 'ZJ000013', 'ZJ000014', 'ZJ000015', 'ZJ000016', 'ZJ000017', 'ZJ000018', 'ZJ000019', 'ZJ000020', 'ZJ000021', 'ZJ000022', 'ZJ000023', 'ZJ000024', 'ZJ000025', 'ZJ000026', 'ZJ000027', 'ZJ000028', 'ZJ000029', 'ZJ000030', 'ZJ000031', 'ZJ000032', 'ZJ000033', 'ZJ000034', 'ZJ000035', 'ZJ000036', 'ZJ000037', 'ZJ000038', 'ZJ000039', 'ZJ000040', 'ZJ000041', 'ZJ000042', 'ZJ000043', 'ZJ000044', 'ZJ000045', 'ZJ000046', 'ZJ000047', 'ZJ000048', 'ZJ000049', 'ZJ000050', 'ZJ000051', 'ZJ000052', 'ZJ000053', 'ZJ000054', 'ZJ000055', 'ZJ000056', 'ZJ000057', 'ZJ000058', 'ZJ000059', 'ZJ000060', 'ZJ000061', 'ZJ000062', 'ZJ000063', 'ZJ000064', 'ZJ000065', 'ZJ000066', 'ZJ000067', 'ZJ000068', 'ZJ000069', 'ZJ000070', 'ZJ000071', 'ZJ000072', 'ZJ000073', 'ZJ000074', 'ZJ000075', 'ZJ000076', 'ZJ000077', 'ZJ000078', 'ZJ000079', 'ZJ000080', 'ZJ000081', 'ZJ000082', 'ZJ

# Train Test split

# Train labels

In [7]:
df_2 = pd.read_csv("train_labels.csv")
df_2.head()

,id,antelope_duiker,bird,blank,civet_genet,hog,leopard,monkey_prosimian,rodent
0,ZJ000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ZJ000001,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,ZJ000002,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ZJ000003,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,ZJ000004,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


# Based on the Site the images are being splited

In [8]:
# Define train and test folders
train_folder = "Train"
test_folder = "Test"
os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

# Shuffle the DataFrame to randomize the order of images
shuffled_df = df.sample(frac=1, random_state=42)

# Split shuffled DataFrame into train and test sets
split_index = int(len(shuffled_df) * 0.8)
train_df = shuffled_df.iloc[:split_index]
test_df = shuffled_df.iloc[split_index:]

# Copy images to train folder
for index, row in train_df.iterrows():
    image_id = row['id']
    image_path = row['filepath']
    index_df_2 = df_2[df_2['id'] == image_id].index.item()
    classes_with_1 = df_2.columns[df_2.iloc[index_df_2] == 1.0].tolist()
    for class_name in classes_with_1:
        destination_folder = os.path.join(train_folder, class_name)
        os.makedirs(destination_folder, exist_ok=True)
        shutil.copy(image_path, destination_folder)

# Copy images to test folder
for index, row in test_df.iterrows():
    image_id = row['id']
    image_path = row['filepath']
    index_df_2 = df_2[df_2['id'] == image_id].index.item()
    classes_with_1 = df_2.columns[df_2.iloc[index_df_2] == 1.0].tolist()
    for class_name in classes_with_1:
        destination_folder = os.path.join(test_folder, class_name)
        os.makedirs(destination_folder, exist_ok=True)
        shutil.copy(image_path, destination_folder)

# Data Loader

### Change with your root dir

In [9]:
ROOT_PATH = "/home/user/classification_2"

In [10]:
def get_dataset_loader(batch_size=256):
  """  Get the dataset loader for training or testing.

    Parameters:
    - data_path (str): Path to the dataset directory.
    - batch_size (int): Batch size for the dataloader.
    - train (bool): Whether to get the training dataset loader or testing dataset loader.

    Returns:
    - dataloader (torch.utils.data.DataLoader): Dataset loader.
    - dataset_size (int): Size of the dataset. """
  
  transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomApply(torch.nn.ModuleList([transforms.ColorJitter(), transforms.GaussianBlur(3)]), p=0.1),
    transforms.Resize(633),
    transforms.CenterCrop(600),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.466), (0.229, 0.224, 0.255)),
    transforms.RandomErasing(p=0.12, value='random')
  ])

  train_imgs = datasets.ImageFolder(os.path.join(ROOT_PATH, 'Train/'), transform=transform)

  print(f"We have {len(train_imgs)} Training Images with {len(train_imgs.classes)} classes")

  val_imgs = datasets.ImageFolder(os.path.join(ROOT_PATH , 'Test/'), transform = transform)

  print(f"We have {len(val_imgs)} Training Images with {len(val_imgs.classes)} classes")


  train_dataloader = DataLoader(train_imgs, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)

  validation_dataloader = DataLoader(val_imgs, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)

  return train_dataloader, validation_dataloader, len(train_imgs), len(val_imgs)


In [11]:
BATCH_SIZE = 32
(train_dataloader, validation_dataloader, len_train_dataloader, len_validation_dataloader) = get_dataset_loader( batch_size=BATCH_SIZE)


We have 13190 Training Images with 8 classes
We have 3298 Training Images with 8 classes


In [12]:
dataloaders_dict = {
  'train': train_dataloader,
  'validation': validation_dataloader
}

dataset_sizes_dict = {
  'train':len_train_dataloader,
  'validation': len_validation_dataloader
}

In [13]:
class_categories = datasets.ImageFolder(os.path.join(ROOT_PATH, 'Train/')).classes
class_categories

['antelope_duiker',
 'bird',
 'blank',
 'civet_genet',
 'hog',
 'leopard',
 'monkey_prosimian',
 'rodent']

# Load the Model

In [15]:
torch.backends.cudnn.benchmark = True

model = models.efficientnet_b7(pretrained=True)

print("model.classifier is => ", model.classifier)

""" model.classifier is =>  Sequential(
  (0): Dropout(p=0.2, inplace=True)
  (1): Linear(in_features=1280, out_features=1000, bias=True)
)
"""

for param in model.parameters():
  param.requires_grad = False
  
num_inputs = model.classifier[1].in_features


""" Now here is where I am adding additional layers to the pre-trained model.

for nn.Linear - note the following
in_features – size of each input sample
out_features – size of each output sample

"""


model.classifier = nn.Sequential(
  nn.Linear(num_inputs, 2048),
  nn.SiLU(), # Sigmoid Weighted Linear Unit
  nn.Dropout(0.2),
  # Note that the last layer is 2048 * Number of Classes
  # Reshape the final layer(s) to have the same number of outputs as the number of classes in the new dataset
  nn.Linear(2048, len(class_categories))
)

model = model.to(device)
print(model.classifier)



model.classifier is =>  Sequential(
  (0): Dropout(p=0.5, inplace=True)
  (1): Linear(in_features=2560, out_features=1000, bias=True)
)
Sequential(
  (0): Linear(in_features=2560, out_features=2048, bias=True)
  (1): SiLU()
  (2): Dropout(p=0.2, inplace=False)
  (3): Linear(in_features=2048, out_features=8, bias=True)
)


In [16]:
criterion = nn.CrossEntropyLoss(label_smoothing=0.11)
criterion = criterion.to(device)
optimizer = optim.AdamW(model.classifier.parameters(), lr=0.001 )

In [17]:
training_history = {'accuracy': [], 'loss':[] }
validation_history = {'accuracy': [], 'loss': []}

In [18]:
def train(model, criterion, optimizer, scheduler, num_epochs=25):
  start_time = time.time()
  
  best_model_weights = copy.deepcopy(model.state_dict())
  best_accuracy = 0.0
  
  for epoch in range(num_epochs):
    print('Running epoch {}/{}'.format(epoch, num_epochs-1))
    print('-' * 10)
    
    """ Within each epoch first configure training or eval mode 
    based on wheather I am in the 'train' or 'validation' phase.
    
    `model.train()` tells your model that you are training the model. So effectively layers like dropout, batchnorm etc.
     which behave different on the train and test procedures know what is going on and hence can behave accordingly.
    
    In below, after I start the for loop for phase, the rest entire code for this train() function runs
    within this for loop and at various stage passing the phase variable to be either 'train' or 
    'validation'
    
    """
    for phase in ['train', 'validation']:
      if phase == 'train':
        model.train()
      else:
        model.eval()
      
      running_loss = 0.0
      running_corrects = 0
      
      """ now start the iteration of the dataloaders_dict variable
      i.e. for both 'train' and 'validation' phases 
      
      Below loop, basically tells that, for every epoch the train_dataloader and then the validation_dataloader is invoked
       which returns input and its corresponding label. 
      
      """
      for inputs, labels in tqdm(dataloaders_dict[phase]):
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        """ for every mini-batch during the training phase, we typically want to explicitly set the gradients 
        to zero before starting to do backpropragation """
        optimizer.zero_grad()
        
        """ Start the forward movement
        tracking history for phase == 'train' 
        set_grad_enabled is a Context-manager that sets gradient calculation to on or off.

        The argument to it is called mode (which is a boolean ), and this mode controls whether to enable grad (True), 
        or disable (False). This can be used to conditionally enable gradients.
        """
        with torch.set_grad_enabled(phase == 'train'):
          predicted_outputs = model(inputs)          
          # print('outputs on which to apply torch.max ', outputs)
          # find the maximum along the rows, use dim=1 to torch.max()
          _, predictions = torch.max(predicted_outputs, 1)
          
          loss = criterion(predicted_outputs, labels)
          
          # Now for the 'train' phase, do backpropagation and update weights with step()
          if phase == 'train':
            loss.backward()
            optimizer.step()
            
        """ Update the running loss and corrects 
        we need to use, loss.item() instead of loss alone in running_loss calculation and averaging. 
        Because loss gives you a grad_function, not a float value. 
        The item() method extracts the loss’s value as a Python float.
        """
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(predictions == labels.data)
        
      if phase == 'train':
        scheduler.step()      
      
      epoch_loss = running_loss / dataset_sizes_dict[phase]
      epoch_accuracy = running_corrects.double() / dataset_sizes_dict[phase]
      
      """ To summarize, in the above code what I did is
      
      1. => multiply each average batch loss with batch-length. 
      The batch-length is inputs.size(0) which gives the number total images in each batch. 
      Essentially I am un-averaging the Batch-Loss

      Do this all the batches inside the batch-running loop.
      And then after that loop, i.e. outside the batch-loop and coming back to my epoch-loop
      
      2. => Divide this accumulated un-averaged Batch-loss from all batches, 
      by the number of samples (len(train_dataloader)) to get the exact train loss average for the epoch """
      
      
      if phase == 'train':
        training_history['loss'].append(epoch_loss)
        training_history['accuracy'].append(epoch_accuracy)
        print(f"Train stats : loss{epoch_loss} , accu{epoch_accuracy}")

      elif phase == 'validation':
        validation_history['loss'].append(epoch_loss)
        validation_history['accuracy'].append(epoch_accuracy)
        print(f"Val stats : loss{epoch_loss} , accu{epoch_accuracy}")
        
      # finally update best_accuracy and deep copy the model
      if phase == 'validation' and epoch_accuracy > best_accuracy:
        best_accuracy = epoch_accuracy
        best_model_weights = copy.deepcopy(model.state_dict())
  
  time_elapsed = time.time() - start_time
  print('Training complete in {:.0f}m {:.0f}s'.format(
    time_elapsed // 60, time_elapsed % 60
  ))
  print('Best Validation Accuracy: {:4f}'.format(best_accuracy))
  
  # loading best model weights
  model.load_state_dict(best_model_weights)
  return model

In [19]:
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.8)
# Change the learning rate based on number of epochs
# Decays the learning rate of each parameter group by gamma every step_size epochs

# Model Traning 

In [22]:
model_trained = train(model, criterion, optimizer, exp_lr_scheduler, num_epochs=100)

Running epoch 0/9
----------


100%|██████████| 413/413 [04:23<00:00,  1.57it/s]


Train stats : loss1.109556605076591 , accu0.7267626990144049


100%|██████████| 104/104 [00:56<00:00,  1.86it/s]


Val stats : loss1.1011057515951126 , accu0.749241964827168
Running epoch 1/9
----------


...

Val stats : loss1.0367547727628792 , accu0.7801697998787144
Running epoch 9/9
----------


100%|██████████| 413/413 [04:16<00:00,  1.61it/s]


Train stats : loss1.0192788618093553 , accu0.7709628506444276


100%|██████████| 104/104 [00:54<00:00,  1.89it/s]

Val stats : loss1.034426861780929 , accu0.7825955124317768
Training complete in 52m 33s
Best Validation Accuracy: 0.782596


### The best accuracy of the model is 0.782 with 10 epochs

In [23]:
torch.save(model_trained.state_dict(), 'EfficientNet_classification.pth')

# Eval()

In [25]:
df_test = pd.read_csv('test_features.csv')
df_test

,id,filepath,site
0,ZJ016488,test_features/ZJ016488.jpg,S0082
1,ZJ016489,test_features/ZJ016489.jpg,S0040
2,ZJ016490,test_features/ZJ016490.jpg,S0040
3,ZJ016491,test_features/ZJ016491.jpg,S0041
4,ZJ016492,test_features/ZJ016492.jpg,S0040
...,...,...,...
4459,ZJ020947,test_features/ZJ020947.jpg,S0086
4460,ZJ020948,test_features/ZJ020948.jpg,S0152
4461,ZJ020949,test_features/ZJ020949.jpg,S0162
4462,ZJ020950,test_features/ZJ020950.jpg,S0055


In [26]:
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])

    image = Image.open(image_path).convert("RGB")
    image = transform(image)
    image = image.unsqueeze(0)  # Add batch dimension
    return image


In [27]:
columns  = class_categories.copy()
columns.insert(0,'id')
columns

['id',
 'antelope_duiker',
 'bird',
 'blank',
 'civet_genet',
 'hog',
 'leopard',
 'monkey_prosimian',
 'rodent']

In [28]:
def evaluation(model):
      
    results = []

    model.eval()  # Set the model to evaluation mode
    model.to(device)
    
    for idx, row in df_test.iterrows():
        image_path = row['filepath']
        image_tensor = preprocess_image(image_path)

        with torch.no_grad():
            image_tensor = image_tensor.to(device)
            output = model(image_tensor)
            probabilities = torch.nn.functional.softmax(output[0], dim=0)          

            result = [row['id']] + probabilities.cpu().numpy().tolist()
            results.append(result)

    return results

In [29]:
results = evaluation(model)

In [30]:
results_df = pd.DataFrame(results, columns=columns)
results_df

,id,antelope_duiker,bird,blank,civet_genet,hog,leopard,monkey_prosimian,rodent
0,ZJ016488,0.064071,0.033285,0.536629,0.063123,0.088092,0.097634,0.043758,0.073408
1,ZJ016489,0.013698,0.039491,0.442567,0.067420,0.119986,0.289670,0.009221,0.017946
2,ZJ016490,0.028315,0.081680,0.378705,0.105270,0.210736,0.165264,0.011434,0.018596
3,ZJ016491,0.006369,0.035977,0.490700,0.017582,0.045365,0.376683,0.016954,0.010370
4,ZJ016492,0.045205,0.563468,0.061892,0.036766,0.040819,0.117830,0.093955,0.040065
...,...,...,...,...,...,...,...,...,...
4459,ZJ020947,0.018612,0.061308,0.733139,0.021737,0.009518,0.080204,0.036685,0.038797
4460,ZJ020948,0.132559,0.335379,0.213071,0.038468,0.002872,0.012272,0.101380,0.163999
4461,ZJ020949,0.008411,0.016088,0.660600,0.019868,0.048072,0.176261,0.004094,0.066606
4462,ZJ020950,0.023402,0.448771,0.131228,0.011011,0.069809,0.285921,0.022248,0.007610


In [31]:
results_df.to_csv('results/submission.csv', index=False) 